In [2]:
import yfinance as yf
from datetime import datetime
import quandl
import stockquotes
import numpy as np
import scipy.stats as st 
import pandas as pd 
import matplotlib.pyplot as plt
import math

%matplotlib inline

In [3]:
def Spot(ticker):
    '''
    Takes a ticker symbol and returns the last price of the asset.
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    Returns:
    S -- Spot price of the asset
    '''
    
    security = stockquotes.Stock(ticker)
    S = security.current_price

    return S    

In [4]:
S = Spot("^GSPC")
print(f'S&P500 last price was ${S}, thus we will look for strike price in the range of ${round(S-300)} - ${round(S+300)}')

S&P500 last price was $3647.49, thus we will look for strike price in the range of $3347 - $3947


In [5]:
def call_data(ticker,expiration):
    '''
    Extracts the strike price and call premium of an option given the ticker symbol of the underlying asset and 
    the expiration date. Returns a list of strike price and call premium.
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset
                  
    Returns:
    data -- list containig pairs of strike price and call premiums available for the underlying in the expiration selected
    '''
    
    security = yf.Ticker(ticker)
    
    if expiration not in security.options:                       #Assertion to verify that expiration exists
        print('Option expiration not available, please verify')
        return
        
    options = security.option_chain(expiration)
    calls = options.calls
    strike_price = calls['strike']
    call_premium = calls['lastPrice']
    
    data = list(zip(strike_price, call_premium))
    data = np.array(data)

    
    return data

In [6]:
call_data('^GSPC','2021-12-16')

array([[ 900.  , 2755.29],
       [1000.  , 2656.54],
       [1225.  , 1956.5 ],
       [1300.  , 2039.26],
       [1375.  , 2143.78],
       [1400.  , 2119.6 ],
       [1500.  , 2160.15],
       [1525.  , 1425.66],
       [1550.  , 1201.5 ],
       [1600.  , 1600.8 ],
       [1625.  , 1375.  ],
       [1650.  , 1054.5 ],
       [1800.  , 1243.1 ],
       [1825.  , 1466.  ],
       [1900.  , 1361.4 ],
       [2000.  , 1638.  ],
       [2175.  ,  779.5 ],
       [2200.  , 1166.64],
       [2250.  , 1311.3 ],
       [2300.  , 1321.8 ],
       [2350.  ,  834.  ],
       [2400.  , 1007.  ],
       [2425.  ,  760.89],
       [2475.  , 1074.55],
       [2500.  , 1205.23],
       [2525.  ,  575.4 ],
       [2550.  ,  877.68],
       [2575.  ,  895.15],
       [2600.  , 1043.62],
       [2625.  ,  920.  ],
       [2650.  ,  909.5 ],
       [2675.  ,  564.9 ],
       [2700.  ,  960.  ],
       [2725.  ,  756.5 ],
       [2750.  ,  966.06],
       [2775.  ,  371.06],
       [2800.  ,  945.42],
 

In [7]:
def Risk_free(ttm):
    '''
    Extracts the most recent US treasury rate given the time to maturity.
    
    Arguments:
    ttm -- Time to maturity in string format. The available options are:
           ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    Returns:
    rf -- Most recent risk free rate for the time to maturity selected
    '''
    yield_curve = quandl.get('USTREASURY/YIELD', authtoken='JMxryiBcRV26o9r5q7uv')
    rf = yield_curve[ttm][-1]/100
    
    return rf    

In [8]:
Risk_free('1 YR')

0.001

In [9]:
def Time_to_maturity(expiration):
    '''
    Calculates the time to maturity in years from today to the option expiration.
    
    Arguments:
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset
                  
    Returns:
    ttm -- Time to maturity in year in float format
    '''
    
    today = datetime.now()
    expiration_d = datetime.strptime(expiration,'%Y-%m-%d')
    datetime_delta = (expiration_d - today).days
    ttm = datetime_delta/365
    
    return ttm
    

In [10]:
Time_to_maturity('2021-12-16')

1.0

In [11]:
def Norm_d1(ticker, expiration, ttm, sigma = 0.1):
    
    '''
    Calculates the expected value of the stock if the option is exercised using risk-adjusted probabilities.
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset  
                  
    ttm -- Time to maturity in string format. The available options are:
           ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    sigma -- implied volatility. Default value to 0.1
    
    Returns:
    N_d1 -- Array of probability to receive the stock given that the stock price is higher than the strike price at 
    expiration
    
    cache -- cache of values needed for Norm_d2() function

    '''
    
    S = Spot(ticker)
    
    data = call_data(ticker,expiration)
    
    K, call_premium = list(zip(*data))
    call_premium = np.array(call_premium)
    
    S = np.array(S)
    K = np.array(K)
    
    r = Risk_free(ttm)
    
    T = Time_to_maturity(expiration)
    
    
    d1 = (np.log(np.divide(S,K))+(r+((np.power(sigma,2)/2))*T))/(sigma*np.sqrt(T))
    N_d1 = st.norm.cdf(d1)
    
    cache = (d1, T, r, S, K, sigma, call_premium)
    
    return N_d1, cache

In [12]:
Norm_d1('^GSPC','2021-12-16','1 YR')

(array([1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 0.99999992, 0.99999984, 0.9999995 , 0.9999985 ,
        0.99999583, 0.9999891 , 0.99998279, 0.99995892, 0.99993785,
        0.99990725, 0.99986339, 0.99980137, 0.99971476, 0.99959533,
        0.99943263, 0.99921358, 0.99892207, 0.99853844, 0.9980391 ,
        0.99739605, 0.99657653, 0.99554266, 0.99425124, 0.99069582,
        0.98831856, 0.98545781, 0.97800939, 0.95413786, 0.91448231,
        0.8556932 , 0.77716043, 0.68180883, 0.57575841, 0.27113864,
        0.19419219]),
 (array([14.05399775, 13.0003926 , 10.97098416, 10.37674995,  9.81585529,
          9.63567023,  8.94574152,  8.7804485 ,  8.61784329,  8.30035631,
          8.14531444,  7.99263972,  7.12252595,  6.98459273,  6.58185374,
          6.06892079,  5.23010595,  5.11581899,  4.89109044,  4.67130137,
  

In [13]:
def Norm_d2(ticker, expiration, ttm, sigma = 0.1):
    '''
    Calculates the probability that the option will be exercised based on risk-adjusted probabilities.
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset  
                  
    ttm -- Time to maturity in string format. The available options are:
           ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    sigma -- implied volatility. Default value to 0.1
    
    Returns:
    N_d2 -- Array of probabilities that the option will end up in-the-money thus excercised for each strike price
    
    cache -- Cache of values needed for Call_price() function

    '''
    
    N_d1, cache = Norm_d1(ticker, expiration, ttm, sigma)
    
    d1, T, r, S, K, sigma, call_premium = cache
    
    d2 = np.subtract(d1,sigma*np.sqrt(T))
    N_d2 = st.norm.cdf(d2)
    
    cache = (N_d1, S, K, r, T, call_premium)
    
    return N_d2, cache

In [14]:
Norm_d2('^GSPC','2021-12-16','1 YR')

(array([1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 1.        , 1.        , 1.        , 1.        ,
        1.        , 0.99999986, 0.99999974, 0.99999917, 0.99999758,
        0.99999338, 0.99998306, 0.99997351, 0.99993798, 0.99990707,
        0.99986261, 0.99979953, 0.99971118, 0.999589  , 0.99942213,
        0.99919695, 0.99889663, 0.99850067, 0.99798439, 0.99731852,
        0.99646877, 0.99539553, 0.99405359, 0.99239212, 0.98787941,
        0.98489964, 0.98134433, 0.97220715, 0.94367247, 0.89775887,
        0.83176613, 0.74621888, 0.64533755, 0.53627522, 0.23904654,
        0.16788645]),
 (array([1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 1.        , 1.        , 1.        , 1.        ,
         1.        , 0.99999992, 0.99999984, 0.9999995 , 0.9999985 ,
         0.99999583, 0

In [15]:
def Call_price(ticker, expiration, ttm, sigma = 0.1):
    
    '''
    Call culates the call price of an option from an underlying asset that doesn't pay dividends using the Black-Scholes
    model
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset  
                  
    ttm -- Time to maturity in string format. The available options are:
           ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    sigma -- implied volatility. Default value to 0.1
    
    Returns:
    C -- Array of calculated call price using the Black-Scholes formula for each strike price
    
    call_premium -- Array of market price of a call option for each strike price

    '''
    
    N_d2, cache = Norm_d2(ticker, expiration, ttm, sigma)
    
    N_d1, S, K, r, T, call_premium = cache
    
    C = np.subtract(np.multiply(S,N_d1),np.multiply(K,np.multiply(np.exp(-r*T),N_d2)))
    
    call_premium = np.array(call_premium)
    
    return C, call_premium

In [16]:
Call_price('^GSPC','2021-12-16','1 YR')

(array([2748.38955015, 2648.48950017, 2423.7143877 , 2348.78935022,
        2273.86431273, 2248.88930023, 2148.98925025, 2124.01423775,
        2099.03922526, 2049.08920027, 2024.11418777, 1999.13917527,
        1849.2891003 , 1824.3140878 , 1749.38905032, 1649.48900037,
        1474.6639185 , 1449.68891097, 1399.7389108 , 1349.78896049,
        1299.83914515, 1249.88967143, 1224.91519304, 1174.96726414,
        1149.99416291, 1125.02199367, 1100.05115525, 1075.08219218,
        1050.11583801, 1025.15306799, 1000.19516221,  975.24377992,
         950.3010456 ,  925.36964694,  900.45294434,  875.55509119,
         850.68116322,  825.83729491,  801.03081993,  751.56622286,
         726.93000968,  702.37525174,  653.57317964,  557.74759407,
         465.60808858,  379.04763132,  300.07657788,  230.46703723,
         171.41190462,   57.6257779 ,   37.43948784]),
 array([2755.29, 2656.54, 1956.5 , 2039.26, 2143.78, 2119.6 , 2160.15,
        1425.66, 1201.5 , 1600.8 , 1375.  , 1054.5 , 1243.

In [17]:
def difference(ticker, expiration, ttm, sigma = 0.1):
    '''
    Implements the difference between the call price calculated by the Black-Scholes model and the market call price for
    each strike price
    
    Arguments:
    ticker -- String that represents the ticker symbol of the underlying asset. Refer to Yahoo Finance website to find the 
              symbol
              
    expiration -- String date in format 'YYYY-MM-DD'. Refer to Yahoo Finance to check list of available expirations for 
                  the underlying asset  
                  
    ttm -- Time to maturity in string format. The available options are:
           ['1 MO', '2 MO', '3 MO', '6 MO', '1 YR', '2 YR', '3 YR', '5 YR', '7 YR', '10 YR', '20 YR', '30 YR']
    
    sigma -- implied volatility. Default value to 0.1

    Returns:
    difference -- Array of differences between Black-Scholes call price and market call price for each strike price
    '''
    
    B_S, call_premium = Call_price(ticker, expiration, ttm, sigma)
    
    difference = np.subtract(B_S, call_premium)
    
    return difference

In [18]:
def vega(ticker, expiration, ttm, sigma=0.1):
    S = Spot(ticker)
    
    T = Time_to_maturity(expiration)
    
    N_d1, cache = Norm_d1(ticker, expiration, ttm, sigma)
    d1, T, r, S, K, sigma, call_premium = cache
    
    
    v = -S*np.sqrt(T)*st.norm.pdf(d1)
    
    return v, sigma
    

In [19]:
vega('^GSPC','2021-12-16','1 YR')

(array([-1.87542121e-40, -2.90271154e-34, -1.06296954e-23, -6.04162588e-21,
        -1.73999140e-18, -1.00373915e-17, -6.10120783e-15, -2.64034449e-14,
        -1.08637964e-13, -1.59347907e-12, -5.70192283e-12, -1.95452499e-11,
        -1.40262966e-08, -3.71087970e-08, -5.70049124e-07, -1.46211158e-05,
        -1.67130899e-03, -3.01864874e-03, -9.29269361e-03, -2.65779798e-02,
        -7.09217291e-02, -1.77257745e-01, -2.73748248e-01, -6.24364310e-01,
        -9.22848101e-01, -1.34527896e+00, -1.93484085e+00, -2.74652896e+00,
        -3.84927665e+00, -5.32813213e+00, -7.28638006e+00, -9.84747829e+00,
        -1.31566575e+01, -1.73820143e+01, -2.27149199e+01, -2.93695657e+01,
        -3.75814825e+01, -4.76048929e+01, -5.97087977e+01, -9.12753099e+01,
        -1.11296328e+02, -1.34498106e+02, -1.91371157e+02, -3.51051036e+02,
        -5.70169462e+02, -8.28666130e+02, -1.08794669e+03, -1.30127993e+03,
        -1.42882136e+03, -1.20856420e+03, -1.00310842e+03]),
 0.1)

In [20]:
def vega(ticker, expiration, ttm, sigma = 0.1):
    S = Spot(ticker)
    print('S =',S)
    
    T = Time_to_maturity(expiration)
    print('T =',T)
    
    data = call_data(ticker, expiration)
    K, call_premium = list(zip(*data))
    K = np.array(K)
    print('K =',K[0])
    
    r = Risk_free(ttm)
    print('r =',r)
    
    A = np.log(S/K) + (r + (sigma**2/2))*T
    print('A =',A)
    
    B = A**2/(2*sigma**2*T)
    print('B =',B)
    
    
    v = S * (np.sqrt(T/(2*math.pi))) * np.exp(-(np.log(S/K) + (r+sigma**2/2)*T)**2/(2*sigma**2*T))
    
    return v

In [21]:
vega('^GSPC','2021-12-16','1 YR')

S = 3647.49
T = 1.0
K = 900.0
r = 0.001
A = [ 1.40539978  1.30003926  1.09709842  1.037675    0.98158553  0.96356702
  0.89457415  0.87804485  0.86178433  0.83003563  0.81453144  0.79926397
  0.71225259  0.69845927  0.65818537  0.60689208  0.5230106   0.5115819
  0.48910904  0.46713014  0.44562393  0.42457052  0.41420774  0.39379886
  0.38374853  0.3737982   0.3639459   0.35418973  0.34452781  0.33495836
  0.32547962  0.31608988  0.30678749  0.29757083  0.28843835  0.27938851
  0.27041984  0.2615309   0.25272027  0.23532852  0.22674478  0.21823409
  0.20142697  0.16863715  0.13688845  0.10611679  0.07626383  0.04727629
  0.01910541 -0.06093729 -0.0862551 ]
B = [9.87574264e+01 8.45051039e+01 6.01812467e+01 5.38384698e+01
 4.81755075e+01 4.64230704e+01 4.00131456e+01 3.85481379e+01
 3.71336115e+01 3.44479574e+01 3.31730737e+01 3.19411448e+01
 2.53651879e+01 2.43922678e+01 2.16603993e+01 1.84158998e+01
 1.36770041e+01 1.30858020e+01 1.19613828e+01 1.09105282e+01
 9.92903442e+00 9.01300643

array([1.87542121e-40, 2.90271154e-34, 1.06296954e-23, 6.04162588e-21,
       1.73999140e-18, 1.00373915e-17, 6.10120783e-15, 2.64034449e-14,
       1.08637964e-13, 1.59347907e-12, 5.70192283e-12, 1.95452499e-11,
       1.40262966e-08, 3.71087970e-08, 5.70049124e-07, 1.46211158e-05,
       1.67130899e-03, 3.01864874e-03, 9.29269361e-03, 2.65779798e-02,
       7.09217291e-02, 1.77257745e-01, 2.73748248e-01, 6.24364310e-01,
       9.22848101e-01, 1.34527896e+00, 1.93484085e+00, 2.74652896e+00,
       3.84927665e+00, 5.32813213e+00, 7.28638006e+00, 9.84747829e+00,
       1.31566575e+01, 1.73820143e+01, 2.27149199e+01, 2.93695657e+01,
       3.75814825e+01, 4.76048929e+01, 5.97087977e+01, 9.12753099e+01,
       1.11296328e+02, 1.34498106e+02, 1.91371157e+02, 3.51051036e+02,
       5.70169462e+02, 8.28666130e+02, 1.08794669e+03, 1.30127993e+03,
       1.42882136e+03, 1.20856420e+03, 1.00310842e+03])

In [22]:
import time

In [23]:
def update_sigma(iterations, ticker, expiration, ttm, learning_rate=0.075):
    
    tic = time.time()
    implied_vol = np.ones(51)
    implied_vol = implied_vol*0.8
    
    
    S = Spot(ticker)
    
    data = call_data(ticker,expiration)
    
    K, call_premium = list(zip(*data))
    call_premium = np.array(call_premium)
    
    S = np.array(S)
    K = np.array(K)
    
    r = Risk_free(ttm)
    
    T = Time_to_maturity(expiration)

    for i in range(0,iterations):
        if i%100 == 0:
            print('iteration #',i)
            print('implied vol beg:',implied_vol[0])

        d1 = (np.log(np.divide(S,K))+(r+((np.power(implied_vol,2)/2))*T))/(implied_vol*np.sqrt(T))
        N_d1 = st.norm.cdf(d1)
        
        d2 = np.subtract(d1,implied_vol*np.sqrt(T))
        N_d2 = st.norm.cdf(d2)
        
        C = np.subtract(np.multiply(S,N_d1),np.multiply(K,np.multiply(np.exp(-r*T),N_d2)))
        if i%100 == 0:        
            print('BS premium =',C[0])
            print('Market premium =',call_premium[0])

#         C, call_premium = Call_price(ticker, expiration, ttm, implied_vol)

        difference = np.subtract(C, call_premium)
        if i%100 == 0:
            print('The difference is:',difference[0])

        v = S*np.sqrt(T)*st.norm.pdf(d1)
        if i%100 == 0:
            print('vega =',v[0])
    
#         v, sigma = vega(ticker, expiration, ttm)
        for j in range(0,len(difference)):
            if difference[j] >= 0:
                difference[j] = difference[j]-learning_rate*v[j]
            else:
                difference[j] = difference[j]+learning_rate*v[j]
            
        if i%100 == 0:
            print('implied vol end:',implied_vol[0])
            print('-'*20)
            print('\n')

    toc = time.time()
    
    print(toc-tic)
    return implied_vol
        

In [ ]:
update_sigma(5000,'^GSPC','2021-12-16','1 YR')

__References:__<p>&nbsp;</p> 

- Tim Worall (2008). The Black-Scholes Formula, FIN-40008 Financial Instruments http://www.timworrall.com/fin-40008/bscholes.pdf

In [24]:
def implied_vol(iterations, ticker, expiration, ttm): 
    
    S = Spot(ticker)
    S = np.array(S)
    
    data = call_data(ticker,expiration)
    K, call_premium = list(zip(*data))
    K = np.array(K)
    call_premium = np.array(call_premium)
      
    r = Risk_free(ttm)
    
    T = Time_to_maturity(expiration)
    
    sigma_ran = np.random.randn(iterations)
    
    imply_vol = []
    difference = []
    
    for i in range(0,len(K)):

        spread_pos = []
        sigma_pos = []
        spread_neg = []
        sigma_neg = []
        
        for j in range(0,len(sigma_ran)):

            d1 = (np.log(np.divide(S,K[i]))+(r+((np.power(sigma_ran[j],2)/2))*T))/(sigma_ran[j]*np.sqrt(T))
            N_d1 = st.norm.cdf(d1)

            d2 = np.subtract(d1,sigma_ran[j]*np.sqrt(T))
            N_d2 = st.norm.cdf(d2)

            C = np.subtract(np.multiply(S,N_d1),np.multiply(K[i],np.multiply(np.exp(-r*T),N_d2)))

            spread = np.subtract(C, call_premium[i])

            if spread >=0:
                spread_pos.append(spread)
                sigma_pos.append(sigma_ran[j])
            else:
                spread_neg.append(spread)
                sigma_neg.append(sigma_ran[j])

        min_dif = np.min(spread_pos)
        index_min_dif = np.argmin(spread_pos)

        sigma_optimun = sigma_pos[index_min_dif]

        imply_vol.append(sigma_optimun)
        difference.append(min_dif)

    
    print(imply_vol)
    print(difference)
    print(len(imply_vol))
    print(len(difference))
     
    return imply_vol, difference

In [25]:
implied_vol(1000,'^GSPC','2021-12-16','1 YR')

[0.662073905477499, 0.6277689890538044, 0.12863435823636146, 0.12863435823636146, 0.0469876278293256, 0.0469876278293256, 0.46791714174361326, 0.10858844487841277, 0.0469876278293256, 0.0469876278293256, 0.0469876278293256, 0.09639548624087349, 0.0884197953112702, 0.0469876278293256, 0.08111858924473546, 0.07421321790914648, 0.0469876278293256, 0.0469876278293256, 0.0469876278293256, 0.0469876278293256, 0.0469876278293256, 0.0469876278293256, 0.0469876278293256, 0.049391736800324444, 0.3299599901970299, 0.018840630469464183, 0.04458139840684949, 0.018840630469464183, 0.018840630469464183, 0.018840630469464183, 0.018840630469464183, 0.018840630469464183, 0.1779316085953488, 0.018840630469464183, 0.2722556135896575, 0.018840630469464183, 0.2974930153931255, 0.018840630469464183, 0.23609313205493435, 0.23609313205493435, 0.018840630469464183, 0.2593898186657767, 0.2562016704633753, 0.24565070397746747, 0.25954925728553996, 0.2512044245244568, 0.24565070397746747, 0.2164466213318908, 0.206

([0.662073905477499,
  0.6277689890538044,
  0.12863435823636146,
  0.12863435823636146,
  0.0469876278293256,
  0.0469876278293256,
  0.46791714174361326,
  0.10858844487841277,
  0.0469876278293256,
  0.0469876278293256,
  0.0469876278293256,
  0.09639548624087349,
  0.0884197953112702,
  0.0469876278293256,
  0.08111858924473546,
  0.07421321790914648,
  0.0469876278293256,
  0.0469876278293256,
  0.0469876278293256,
  0.0469876278293256,
  0.0469876278293256,
  0.0469876278293256,
  0.0469876278293256,
  0.049391736800324444,
  0.3299599901970299,
  0.018840630469464183,
  0.04458139840684949,
  0.018840630469464183,
  0.018840630469464183,
  0.018840630469464183,
  0.018840630469464183,
  0.018840630469464183,
  0.1779316085953488,
  0.018840630469464183,
  0.2722556135896575,
  0.018840630469464183,
  0.2974930153931255,
  0.018840630469464183,
  0.23609313205493435,
  0.23609313205493435,
  0.018840630469464183,
  0.2593898186657767,
  0.2562016704633753,
  0.24565070397746747,


In [26]:
def implied_vol2(iterations, ticker, expiration, ttm):  
    
    S = Spot(ticker)
    S = np.array(S)
    
    data = call_data(ticker,expiration)
    K, call_premium = list(zip(*data))
    K = np.array(K)
    call_premium = np.array(call_premium)
      
    r = Risk_free(ttm)
    
    T = Time_to_maturity(expiration)
    
    sigma_ran = np.random.randn(iterations)
    
    imply_vol = []
    difference = []
    print('done')
    
    for i in range(0,len(K)):

        spread_pos = []
        sigma_pos = []
        spread_neg = []
        sigma_neg = []

        for j in range(0,len(sigma_ran)):

            d1 = (np.log(np.divide(S,K[i]))+(r+((np.power(sigma_ran[j],2)/2))*T))/(sigma_ran[j]*np.sqrt(T))
            N_d1 = st.norm.cdf(d1)

            d2 = np.subtract(d1,sigma_ran[j]*np.sqrt(T))
            N_d2 = st.norm.cdf(d2)

            C = np.subtract(np.multiply(S,N_d1),np.multiply(K[i],np.multiply(np.exp(-r*T),N_d2)))

            spread = np.subtract(C, call_premium[i])

            if spread >=0:
                spread_pos.append(spread)
                sigma_pos.append(sigma_ran[j])
            else:
                spread_neg.append(spread)
                sigma_neg.append(sigma_ran[j])

        min_dif = np.min(spread_pos)
        index_min_dif = np.argmin(spread_pos)

        max_dif = np.max(spread_neg)
        index_max_dif = np.argmax(spread_neg)

        if abs(min_dif) < abs(max_dif):
            sigma_optimun = sigma_pos[index_min_dif]
            spr = min_dif
        else:
            sigma_optimun = sigma_neg[index_max_dif]
            spr = max_dif

        imply_vol.append(sigma_optimun)
        difference.append(spr)

    
    print(imply_vol)
    print(difference)
    print(len(imply_vol))
    print(len(difference))
     
#     return imply_vol, difference

In [28]:
implied_vol2(10000,'^GSPC','2021-12-16','1 YR')

done
[0.6587667987698821, 0.6254170778194428, 0.09924062852604029, 0.12920183157169582, 0.09924062852604029, 0.1186957207053442, 0.4637817762824479, 0.10781041695857263, 0.10427005282993146, 0.09924062852604029, 0.09834142583396892, 0.09924062852604029, 0.08618977116438942, 0.08658478494890053, 0.0805912261996791, 0.07331000127218314, 0.06398887109147304, 0.0628333947919966, 0.0615190194407895, 0.05604468779767345, 0.0558156792351565, 0.012818374886678278, 0.052701315517563144, 0.04726962740422134, 0.3194713949405582, 0.012818374886678278, 0.044566325430353555, 0.012818374886678278, 0.041812385609151785, 0.04062677617838884, 0.0388397688958646, 0.03824289417704019, 0.17648759972313302, 0.03712411972602887, 0.27170627395222063, -0.022221380996873877, 0.297057173771888, 0.031773883505532466, 0.22682379133105018, 0.23205839576809476, 0.027244924112675484, 0.25806105036625115, 0.2545769100851862, 0.24190123762583302, 0.2594972258483154, 0.2513352928543284, 0.24249540460357355, 0.2143815831